In [1]:
import os
from pathlib import Path

In [2]:
# Domain Contract
os.environ['HADRON_PM_PATH'] = Path('${PWD}/metadata/contracts/workshop').as_posix() 

# local data stores
os.environ['HADRON_DEFAULT_PATH'] = Path('${PWD}/metadata/data/workshop').as_posix()

## Setup Component

In [3]:
from ds_discovery import Wrangle, Transition, Commons

In [4]:
wr = Wrangle.from_env("intel_service", has_contract=False)

In [5]:
wr.setup_bootstrap(domain='Telecom')
contract = Transition.from_env("intel_service").get_persist_contract()
wr.set_source_contract(contract)
wr.set_persist()

In [6]:
df = wr.load_source_canonical()

### dates

In [7]:
df["duration"] =  wr.tools.correlate_date_diff(df,units='ns',first_date = "creationDate", second_date = "modifiedDate", column_name="duration")

In [8]:
df["modifiedDatePlus"] =  wr.tools.correlate_dates(df, header='modifiedDate', offset={'hours': 1}, column_name="modifiedDatePlus")

In [9]:
df["durationPlus"] =  wr.tools.correlate_date_diff(df,units='ns',first_date = "creationDate", second_date = "modifiedDatePlus", column_name="durationPlus")

In [10]:
wr.canonical_report(Commons.filter_columns(df, regex='Date'))

,Attributes (4),dType,%_Null,%_Dom,Count,Unique,Observations
0,creationDate,"datetime64[ns, UTC]",0.0%,0.0%,5000,4999,max=2023-02-16 06:13:51.686000+00:00 | min=2023-02-15 18:59:23.338000+00:00 | yr mean= 2023
1,modifiedDate,"datetime64[ns, UTC]",0.0%,0.0%,5000,4999,max=2023-02-16 06:13:51.686000+00:00 | min=2023-02-15 18:59:23.401000+00:00 | yr mean= 2023
2,modifiedDatePlus,"datetime64[ns, UTC]",0.0%,0.0%,5000,4999,max=2023-02-16 07:13:51.686000+00:00 | min=2023-02-15 19:59:23.401000+00:00 | yr mean= 2023
3,processingDate,"datetime64[ns, UTC]",2.3%,2.3%,5000,4887,max=2023-02-16 06:14:16.601000+00:00 | min=2023-02-15 18:59:20.377000+00:00 | yr mean= 2023


### selection

In [11]:
selection = [wr.tools.select2dict(column='status', condition="@ == 'COMPLETED'")]
df['is_completed'] = wr.tools.correlate_selection(df, selection=selection, action=1, default_action=0, column_name='is_completed')

In [12]:
selection = [wr.tools.s2d(column='requestor', condition="@ != 'DSM'")]
action = wr.tools.a2d(method='@constant', value='other')
default = wr.tools.a2d(method='@header', header='requestor')

df['requestor_type'] = wr.tools.correlate_selection(df, selection=selection, action=action, default_action=default, column_name='requestor_type')

In [13]:
wr.canonical_report(Commons.filter_columns(df, headers=['is_completed'], regex=['requestor']))

,Attributes (3),dType,%_Null,%_Dom,Count,Unique,Observations
0,is_completed,int64,0.0%,61.2%,5000,2,max=1 | min=0 | mean=0.39 | dominant=0
1,requestor,object,2.7%,59.3%,5000,289,Sample: DSM | davisamue | gabisra | sophabdulsa | solomoabd
2,requestor_type,object,0.0%,59.3%,5000,2,Sample: DSM | other


### encoding

In [14]:
df_encoded = wr.tools.model_encode_one_hot(df, headers=['status', 'requestor_type'], drop_first=True, column_name="encode_onehot")

In [19]:
# K-1 one hot encoding
wr.canonical_report(Commons.filter_columns(df_encoded, regex=['status', 'requestor_type']))

,Attributes (4),dType,%_Null,%_Dom,Count,Unique,Observations
0,requestor_type_other,uint8,0.0%,59.3%,5000,2,
1,status_FAILED,uint8,0.0%,60.7%,5000,2,
2,status_INPROGRESS,uint8,0.0%,99.5%,5000,2,
3,status_PENDING,uint8,0.0%,100.0%,5000,2,


In [15]:
df_encoded = wr.tools.model_encode_interger(df_encoded, headers=['reason'], column_name="encode_nominal")

In [16]:
# as the tail has small counts, reduce them to a single category
rank = ['SERUPD', 'COS', 'RESETPIN', 'RVAS']
df_encoded = wr.tools.model_encode_interger(df_encoded, headers=['subType', 'type'], ranking=rank, column_name="encode_ordinal")

In [17]:
# all other categories have been grouped under 4
df_encoded['type'].value_counts()

0    3030
1    1864
4     106
Name: type, dtype: int64

In [18]:
# ordinal/nominal
wr.canonical_report(Commons.filter_columns(df_encoded, headers=['reason', 'type', 'subType']))

,Attributes (3),dType,%_Null,%_Dom,Count,Unique,Observations
0,reason,int64,0.0%,94.1%,5000,8,max=7 | min=0 | mean=0.15 | dominant=0
1,subType,int64,0.0%,60.5%,5000,3,max=4 | min=2 | mean=2.49 | dominant=2
2,type,int64,0.0%,60.6%,5000,3,max=4 | min=0 | mean=0.46 | dominant=0
